In [1]:
from IPython.display import display
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [2]:
data_dir = '../core/dataset'
pd.options.display.max_colwidth = None

## Load & Format Data

# Prescriber Drug Dataset

In [21]:
prescriber_drug_names_df = pd.read_csv(f'{data_dir}/input/Medicare_Part_D_Prescribers_by_Provider_and_Drug_2019.csv')
prescriber_drug_names_df['GE65_Tot_Drug_Cst'].fillna(-1, inplace=True)

In [26]:
prescriber_drug_names_df.isna().sum(axis=0)

Prscrbr_NPI              0
Prscrbr_Last_Org_Name    0
Prscrbr_First_Name       0
Prscrbr_Type             2
Brnd_Name                0
Gnrc_Name                0
Tot_Drug_Cst             0
GE65_Tot_Drug_Cst        0
dtype: int64

# Drug Rating Dataset

In [4]:
drug_ratings_train_df = pd.read_csv(f'{data_dir}/input/drugsComTrain_raw.tsv', sep='\t')
drug_ratings_test_df = pd.read_csv(f'{data_dir}/input/drugsComTest_raw.tsv', sep='\t')

drug_ratings_df = pd.concat((drug_ratings_train_df, drug_ratings_test_df))
drug_ratings_df.drop(['Unnamed: 0', 'usefulCount'], axis=1, inplace=True)
drug_ratings_df.dropna(inplace=True)
drug_ratings_df = drug_ratings_df[~drug_ratings_df['condition'].str.match('\d*</span>')]
drug_ratings_df.iloc[:2]

,drugName,condition,review,rating,date
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil""",9.0,"May 20, 2012"
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective.""",8.0,"April 27, 2010"


# Drug Information

### Get all rows for prescriber drugs that have ratings

In [5]:
brnd_name_overlap = prescriber_drug_names_df['Brnd_Name'].isin(set(drug_ratings_df['drugName']))
gnrc_name_overlap = prescriber_drug_names_df['Gnrc_Name'].isin(set(drug_ratings_df['drugName']))

filtered_prescriber_drug_names_df = prescriber_drug_names_df[brnd_name_overlap | gnrc_name_overlap]
filtered_prescriber_drug_names_df.index.name = 'drug_id'

### Get list of unique prescribed drugs

In [6]:
unique_drug_names_df = filtered_prescriber_drug_names_df[['Brnd_Name', 'Gnrc_Name']].drop_duplicates().reset_index()
unique_drug_names_df.drop('drug_id', axis=1, inplace=True)
unique_drug_names_df.index.name = 'drug_id'

In [7]:
unique_drug_names_df

,Brnd_Name,Gnrc_Name
drug_id,,
0,Azithromycin,Azithromycin
1,Carvedilol,Carvedilol
2,Cefdinir,Cefdinir
3,Clopidogrel,Clopidogrel Bisulfate
4,Eliquis,Apixaban
...,...,...
1376,Cesamet,Nabilone
1377,Temsirolimus,Temsirolimus
1378,Solodyn,Minocycline Hcl


A generic name can have multiple brand names associated with it; we will use generic name as the drug identifier.

In [47]:
unique_drug_names_df[unique_drug_names_df['Gnrc_Name'] == 'Azithromycin']

,Brnd_Name,Gnrc_Name
drug_id,,
0,Azithromycin,Azithromycin
714,Azasite,Azithromycin
1203,Zithromax,Azithromycin


### Write Formatted Dataset

In [48]:
unique_drug_names_df.to_csv(f'{data_dir}/output/drug-ids.tsv', index=False, sep='\t')

# Prescriber Information

### Extract prescriber entities

In [49]:
prescriber_entities_df = filtered_prescriber_drug_names_df[['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name']].drop_duplicates()

### Extract Prescriber Type

In [158]:
prescriber_type_df = filtered_prescriber_drug_names_df[['Prscrbr_NPI', 'Prscrbr_Type']]

In [164]:
unique_prescriber_type_df = prescriber_type_df['Prscrbr_Type'].drop_duplicates()

### Extract Prescriber Drug relation

In [50]:
prescriber_drug_cost_edge_df = filtered_prescriber_drug_names_df[['Prscrbr_NPI', 'Gnrc_Name', 'Tot_Drug_Cst', 'GE65_Tot_Drug_Cst']]

In [34]:
filtered_prescriber_drug_names_df.iloc[:3]

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_Type,Brnd_Name,Gnrc_Name,Tot_Drug_Cst,GE65_Tot_Drug_Cst
drug_id,,,,,,,,
3,1003000126,Enkeshafi,Ardalan,Internal Medicine,Azithromycin,Azithromycin,132.57,-1.00
4,1003000126,Enkeshafi,Ardalan,Internal Medicine,Carvedilol,Carvedilol,118.53,118.53
5,1003000126,Enkeshafi,Ardalan,Internal Medicine,Cefdinir,Cefdinir,203.03,-1.00


### Write Formatted Dataset

In [53]:
prescriber_entities_df.to_csv(f'{data_dir}/output/prescriber.tsv', index=False, sep='\t')
prescriber_type_df.to_csv(f'{data_dir}/output/prescriber-type.tsv', index=False, sep='\t')
unique_prescriber_type_df.to_csv(f'{data_dir}/output/unique_prescriber_type.tsv', index=False, sep='\t')
prescriber_drug_cost_edge_df.to_csv(f'{data_dir}/output/prescriber-drug.tsv', index=False, sep='\t')

In [169]:
prescriber_drug_cost_edge_df

,Prscrbr_NPI,Gnrc_Name,Tot_Drug_Cst,GE65_Tot_Drug_Cst
drug_id,,,,
3,1003000126,Azithromycin,132.57,-1.00
4,1003000126,Carvedilol,118.53,118.53
5,1003000126,Cefdinir,203.03,-1.00
6,1003000126,Clopidogrel Bisulfate,172.04,172.04
8,1003000126,Apixaban,7234.67,-1.00
...,...,...,...,...
25401863,1992999833,Carbidopa/Levodopa,19319.43,19319.43
25401866,1992999874,Bumetanide,433.17,-1.00
25401867,1992999874,Carvedilol,182.25,-1.00


# Disease Info

### Get unique disease/condition names from ratings file

In [8]:
conditions_df = drug_ratings_df['condition'].drop_duplicates()
conditions_df.to_csv(f'{data_dir}/output/disease-names.tsv', index=False, sep='\t')

# Rating/Review Info

In [8]:
ratings_with_gnrc_df = pysqldf("select distinct a.*, b.Gnrc_Name from drug_ratings_df as a join unique_drug_names_df as b on a.drugName = b.Brnd_Name or a.drugName = b.Gnrc_Name")
ratings_with_gnrc_df.index.name = "review_id"
ratings_with_gnrc_df.iloc[:2]

,drugName,condition,review,rating,date,Gnrc_Name
review_id,,,,,,
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil""",9.0,"May 20, 2012",Valsartan
1,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard erections however experianced headache, lower bowel preassure. 3rd day erections would wake me up &amp; hurt! Leg/ankles aches severe lower bowel preassure like you need to go #2 but can&#039;t! Enjoyed the initial rockhard erections but not at these side effects or $230 for months supply! I&#039;m 50 &amp; work out 3Xs a week. Not worth side effects!""",2.0,"November 28, 2015",Tadalafil


### Get drug-disease (used_for) edges

In [58]:
used_for_df = ratings_with_gnrc_df[['Gnrc_Name', 'condition']].drop_duplicates()

### Get review vertices

In [59]:
reviews_df = ratings_with_gnrc_df[['review', 'rating', 'date']]
reviews_df.index.name = "review_id"

print(reviews_df.shape)
reviews_df.iloc[:2]

(91047, 3)


,review,rating,date
review_id,,,
0,"""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil""",9.0,"May 20, 2012"
1,"""2nd day on 5mg started to work with rock hard erections however experianced headache, lower bowel preassure. 3rd day erections would wake me up &amp; hurt! Leg/ankles aches severe lower bowel preassure like you need to go #2 but can&#039;t! Enjoyed the initial rockhard erections but not at these side effects or $230 for months supply! I&#039;m 50 &amp; work out 3Xs a week. Not worth side effects!""",2.0,"November 28, 2015"


### Get review edges

In [60]:
# review drug edge (has_review)
review_drug_df = ratings_with_gnrc_df['Gnrc_Name']

# review disease edge (associated with)
review_disease_df = ratings_with_gnrc_df['condition']

print(review_drug_df)
print(review_disease_df)

review_id
0                            Valsartan
1                            Tadalafil
2                         Aripiprazole
3                        Levetiracetam
4                           Topiramate
                     ...              
91042             Varenicline Tartrate
91043                    Levetiracetam
91044                     Levofloxacin
91045                      Armodafinil
91046    Desogestrel-Ethinyl Estradiol
Name: Gnrc_Name, Length: 91047, dtype: object
review_id
0        Left Ventricular Dysfunction
1        Benign Prostatic Hyperplasia
2                     Bipolar Disorde
3                            Epilepsy
4                 Migraine Prevention
                     ...             
91042               Smoking Cessation
91043                        Epilepsy
91044                      Bronchitis
91045                      Narcolepsy
91046                   Birth Control
Name: condition, Length: 91047, dtype: object


### Write Formatted Data

In [61]:
used_for_df.to_csv(f'{data_dir}/output/drug-disease.tsv', index=False, sep='\t')
reviews_df.to_csv(f'{data_dir}/output/review.tsv', index=True, sep='\t')
review_drug_df.to_csv(f'{data_dir}/output/drug-review.tsv', index=True, sep='\t')
review_disease_df.to_csv(f'{data_dir}/output/disease-review.tsv', index=True, sep='\t')

# Exploratory Analysis

In [9]:
prescriber_distinct_df = prescriber_drug_names_df.sample(20000)

In [10]:
prescriber_distinct_df

,Prscrbr_NPI,Prscrbr_Last_Org_Name,Prscrbr_First_Name,Prscrbr_Type,Brnd_Name,Gnrc_Name,Tot_Drug_Cst,GE65_Tot_Drug_Cst
23236708,1912326497,Thottam,Justin,Physical Medicine and Rehabilitation,Tizanidine Hcl,Tizanidine Hcl,1378.47,-1.00
3365994,1134164718,Bird,C,Family Practice,Celecoxib,Celecoxib,1218.57,1218.57
12130026,1477702702,Robinson,Tanisha,Pharmacist,Veo Insulin Syringe,"Syringe-Needle,insulin,0.5 Ml",1063.13,-1.00
19301432,1750776126,Deal,Amanda,Family Practice,Diclofenac Sodium,Diclofenac Sodium,561.34,561.34
23726385,1932181831,Grenadier,Mark,Oral Surgery (Dentist only),Amoxicillin-Clavulanate Potass,Amoxicillin/Potassium Clav,140.68,140.68
...,...,...,...,...,...,...,...,...
5160373,1205242286,Bashar,Amir,Family Practice,Furosemide,Furosemide,82.89,-1.00
6474057,1255395935,Zinger,Yevgeniya,Internal Medicine,Lisinopril,Lisinopril,931.26,-1.00
2216652,1083801518,Takata,Jay,Gastroenterology,Prednisone,Prednisone,230.81,-1.00
4317694,1174008643,Powell,Leneal,Dentist,Hydrocodone-Acetaminophen,Hydrocodone/Acetaminophen,116.02,-1.00


In [23]:
prescriber_distinct_df = prescriber_distinct_df.replace(-1, 0)

In [26]:
prescriber_distinct_df_1 = pysqldf("select Prscrbr_NPI, Gnrc_Name, avg(Tot_Drug_Cst) as avg_drug_cost, avg(GE65_Tot_drug_Cst) as avg_drug_cost_ge65 from prescriber_distinct_df group by Prscrbr_NPI, Gnrc_Name")

In [27]:
prescriber_distinct_df_1

,Prscrbr_NPI,Gnrc_Name,avg_drug_cost,avg_drug_cost_ge65
0,1003001884,Dicyclomine Hcl,260.78,0.00
1,1003003856,Carvedilol,212.66,212.66
2,1003014705,Insulin Detemir,41771.25,22105.59
3,1003017021,Ondansetron,392.15,147.87
4,1003025875,Imipramine Hcl,728.14,728.14
...,...,...,...,...
19995,1992972558,Tiotropium Bromide,14804.25,0.00
19996,1992985634,Hydrocodone/Acetaminophen,114.96,0.00
19997,1992990212,"Insulin Regular, Human",16532.56,0.00
19998,1992992440,Fluticasone Propion/Salmeterol,22565.11,22565.11


In [29]:
prescriber_distinct_df_1.to_csv(f"{data_dir}/output/Prescriber_Average_Drug_Costs.tsv", sep='\t')

In [16]:
low_rated_drugs_df = pysqldf("select Gnrc_Name, condition, avg(rating) as avg_rating from ratings_with_gnrc_df where rating <= 5.0 group by Gnrc_Name, condition")

In [12]:
low_rated_drugs_df

,Gnrc_Name,condition,avg_rating
0,Abacavir/Dolutegravir/Lamivudi,HIV Infection,3.750000
1,Abatacept,Rheumatoid Arthritis,2.625000
2,Abatacept/Maltose,Rheumatoid Arthritis,2.625000
3,Abiraterone Acetate,Prostate Cance,4.000000
4,Abobotulinumtoxina,acial Wrinkles,5.000000
...,...,...,...
1726,Zolpidem Tartrate,Insomnia,2.652632
1727,Zonisamide,Benign Essential Trem,1.000000
1728,Zonisamide,Migraine Prevention,3.000000
1729,Zonisamide,Seizures,2.187500


In [30]:
joined_prescriber_low_ratings_df = pysqldf("select b.*, a.condition, a.avg_rating from low_rated_drugs_df as a join prescriber_distinct_df_1 as b on lower(a.Gnrc_Name) = lower(b.Gnrc_Name)")

In [32]:
sorted_joined_prescriber_df = pysqldf("select * from joined_prescriber_low_ratings_df order by avg_drug_cost desc")

In [33]:
sorted_joined_prescriber_df

,Prscrbr_NPI,Gnrc_Name,avg_drug_cost,avg_drug_cost_ge65,condition,avg_rating
0,1659438562,Adalimumab,1376681.35,638024.58,Ankylosing Spondylitis,2.250000
1,1659438562,Adalimumab,1376681.35,638024.58,Crohn's Disease,2.642857
2,1659438562,Adalimumab,1376681.35,638024.58,"Crohn's Disease, Acute",2.000000
3,1659438562,Adalimumab,1376681.35,638024.58,"Crohn's Disease, Maintenance",2.142857
4,1659438562,Adalimumab,1376681.35,638024.58,Hidradenitis Suppurativa,1.000000
...,...,...,...,...,...,...
74501,1134446891,Lorazepam,7.27,0.00,Insomnia,1.333333
74502,1134446891,Lorazepam,7.27,0.00,Nausea/Vomiting,2.714286
74503,1134446891,Lorazepam,7.27,0.00,Panic Disorde,3.833333
74504,1134446891,Lorazepam,7.27,0.00,Sedation,4.400000


In [ ]:
sorted_joined_prescriber_df